### Step 1: Load modules and functions

In [2]:
import xarray as xr
import h5py
from brainio.assemblies import NeuronRecordingAssembly
from pynwb import NWBHDF5IO, NWBFile
from pynwb.base import Images
from pynwb.image import RGBImage, ImageSeries
import glob, os, yaml, pynwb
import pytz  # This is required to handle timezone conversions
from datetime import datetime
from uuid import uuid4
import numpy as np
import scipy.io
import os, glob, json
import pandas as pd
from pynwb.file import Subject
import logging, sys, re
from PIL import Image
import shutil

cwd = os.getcwd()
sys.path.append(os.path.dirname(cwd))
root_dir        = '/braintree/home/aliya277/inventory_new'
df = pd.read_excel( os.path.dirname(cwd)+'/pico_inventory.xlsx' , sheet_name='Sheet2')


In [25]:
def update_sheet(df, exp_path, text):
    imageset = os.path.basename(exp_path).split('.')[0].split('_')[1:]
    if len(imageset) == 1: imageset = imageset[0]
    elif len(imageset) > 1: imageset = '_'.join(imageset)
    mask = df['ImageSet'] == imageset
    index = df.index[mask].tolist()[0]
    df.at[index, 'StimulusSet'] = text

def extract_number(filename):
    # Extract the number from the filename and return it as an integer
    match = re.search(r'\d+', filename)
    return int(match.group()) if match else 0

def update_exp_nwb(experiment_path, list_images_sorted, stimpath, count_1, check_image_order=True):

    combined_nwb = [x for x in os.listdir(experiment_path) if x.endswith('combined.nwb')]
    combined_nwb_test = [x for x in os.listdir(experiment_path) if x.endswith('combined_test.nwb')]
    combined_nwb_train = [x for x in os.listdir(experiment_path) if x.endswith('combined_train.nwb')]

    combined = False
    train    = False
    test     = False
    if len(combined_nwb) != 0: 
        # ------------------------------------------------------------------------------ 
        # Load combined nwb file.
        # ------------------------------------------------------------------------------ 
        io = NWBHDF5IO(os.path.join(experiment_path, combined_nwb[0]), "a") 
        combined_nwb = io.read()
        try: 
            combined_nwb.stimulus_template['StimulusSet']
            print('Simulus Set already exists in combined.')
            display(combined_nwb)
            combined = True
        except: pass

        n_stimuli = []
        for scratch in list(combined_nwb.scratch):
            if scratch.startswith('QualityCheckedPSTH'):
                n_stimuli.append(combined_nwb.scratch[scratch][:].shape[0])
        assert all(element == n_stimuli[0] for element in n_stimuli) == True, 'Number of Stimuli are not consistent over the PSTH!'
        n_stimuli = n_stimuli[0]
        
        # ------------------------------------------------------------------------------ 
        # Load combined_train nwb file.
        # ------------------------------------------------------------------------------ 
        io_train = NWBHDF5IO(os.path.join(experiment_path, combined_nwb_train[0]), "a") 
        combined_nwb_train = io_train.read()
        try: 
            combined_nwb_train.stimulus_template['StimulusSetTrain']
            print('Simulus Set already exists in train.')
            # display(combined_nwb_train)
            train = True
        except: pass

        n_stimuli_train = []
        for scratch in list(combined_nwb_train.scratch):
            if scratch.startswith('QualityCheckedPSTH'):
                n_stimuli_train.append(combined_nwb_train.scratch[scratch][:].shape[0])
        assert all(element == n_stimuli_train[0] for element in n_stimuli_train) == True, 'Number of Stimuli are not consistent over the PSTH!'
        n_stimuli_train = n_stimuli_train[0]


        # ------------------------------------------------------------------------------ 
        # Load combined_test nwb file.
        # ------------------------------------------------------------------------------ 
        io_test = NWBHDF5IO(os.path.join(experiment_path, combined_nwb_test[0]), "a") 
        combined_nwb_test = io_test.read()
        try: 
            combined_nwb_test.stimulus_template['StimulusSetTest']
            print('Simulus Set already exists in test.')
            test = True
        except: pass

        n_stimuli_test = []
        for scratch in list(combined_nwb_test.scratch):
            if scratch.startswith('QualityCheckedPSTH'):
                n_stimuli_test.append(combined_nwb_test.scratch[scratch][:].shape[0])
        assert all(element == n_stimuli_test[0] for element in n_stimuli_test) == True, 'Number of Stimuli are not consistent over the PSTH!'
        n_stimuli_test = n_stimuli_test[0]
        # print(n_stimuli, n_stimuli_train, n_stimuli_test)

        # ------------------------------------------------------------------------------ 
        # Create StimulusSets.
        # ------------------------------------------------------------------------------ 
        if combined == False or train == False or test == False: 
            list_images = []
            list_images_train = []
            list_images_test = []
            for temp, image in enumerate(list_images_sorted):
                image_counter = temp
                if count_1 ==1: image_counter = image_counter+1

                expected_image_name = f'im{image_counter}'
                if check_image_order:
                    if image.split(".")[0] != expected_image_name and image.split(".")[0] !=f'{image_counter}' and image.split(".")[0] !=f'image{image_counter}' and image.split(".")[0] !=f'im{image_counter}_scrambled' and image.split(".")[0] !=f'{image_counter}':
                            print(f'Image names do not increase with +1!!! Expected: {expected_image_name} or image{image_counter} or {image_counter}, Found: {image.split(".")[0]}')

                path = os.path.join(stimpath, image)
                img = Image.open(path)  # an example image

                image_file_name = f'exp_{experiment_name}_{temp}.png'
                nwb_image = RGBImage(
                    name= image_file_name,
                    data=np.array(img.convert("RGB")),
                    resolution=0.0,
                    description= f"StimulusID = {temp}",
                )
                list_images.append(nwb_image)
                if temp < n_stimuli_train: list_images_train.append(nwb_image)
                if temp >= n_stimuli_train: list_images_test.append(nwb_image)
            
            assert n_stimuli == len(list_images), 'Number of Stimuli does not match number of Images!'

        # ------------------------------------------------------------------------------ 
        # Create nwb Images and append.
        # ------------------------------------------------------------------------------ 
        text=['']
        if combined == False: 
            all_images = Images(
                name=f'StimulusSet',
                images= list_images ,
                description= f"This list comprises images forming the stimulus set. \
                Each image in the list has a stimulusID in its description, corresponding to the stimulusID \
                in the PSTHs. If the filenames of these images contain numbers starting from 0 and increasing \
                sequentially, they correspond the StimulusID as well.",
            )
            try: 
                combined_nwb.add_stimulus_template(timeseries=all_images, use_sweep_table=False) 
                print(f"Added StimulusSet to combined nwb.")
                # display(combined_nwb)
                io.write(combined_nwb)
                io.close()  
                text.append('StimulusSet added to combined nwb.')
            except Exception as error: 
                print("An error occurred:", error) 
                io.close()  
                text.append(error)

        if train == False:
            train_images = Images(
                name=f'StimulusSetTrain',
                images= list_images_train ,
                description= f"This list comprises images forming the stimulus set for this training set. \
                Each image in the list has a stimulusID in its description, corresponding to the stimulusID \
                in the PSTHs.",
            )
            try: 
                combined_nwb_train.add_stimulus_template(timeseries=train_images, use_sweep_table=False)  
                print(f"Added StimulusSet to train nwb.")
                # display(combined_nwb_train)
                io_train.write(combined_nwb_train)
                io_train.close()    
                text.append('StimulusSet added to train nwb.')
            except Exception as error: 
                print("An error occurred:", error) 
                io_train.close() 
                text.append(error)

        if test == False:
            test_images = Images(
                name=f'StimulusSetTest',
                images= list_images_test ,
                description= f"This list comprises images forming the stimulus set for this test set. \
                Each image in the list has a stimulusID in its description, corresponding to the stimulusID \
                in the PSTHs.",
            )
            try: 
                combined_nwb_test.add_stimulus_template(timeseries=test_images, use_sweep_table=False) 
                print(f"Added StimulusSet to test nwb.")
                # display(combined_nwb_test)
                io_test.write(combined_nwb_test)
                io_test.close()  
                text.append('StimulusSet added to test nwb.')
            except Exception as error: 
                print("An error occurred:", error)    
                io_test.close() 
                text.append(error)

        # ------------------------------------------------------------------------------ 
        # update excel.
        # ------------------------------------------------------------------------------ 
        if text==['']: text = 'StimulusSet created and added to nwb files.'
        update_sheet(df, experiment_path, text)
        
def update_exp_nwb_movies(experiment_path, list_images_sorted, count_1, check_image_order=True):

    combined_nwb = [x for x in os.listdir(experiment_path) if x.endswith('combined.nwb')]
    combined_nwb_test = [x for x in os.listdir(experiment_path) if x.endswith('combined_test.nwb')]
    combined_nwb_train = [x for x in os.listdir(experiment_path) if x.endswith('combined_train.nwb')]

    combined = False
    train    = False
    test     = False
    if len(combined_nwb) != 0: 
        # ------------------------------------------------------------------------------ 
        # Load combined nwb file.
        # ------------------------------------------------------------------------------ 
        io = NWBHDF5IO(os.path.join(experiment_path, combined_nwb[0]), "a") 
        combined_nwb = io.read()
        try: 
            combined_nwb.stimulus_template['StimulusSet']
            print('Simulus Set already exists in combined.')
            #display(combined_nwb)
            combined = True
        except Exception as error: print(error)

        n_stimuli = []
        for scratch in list(combined_nwb.scratch):
            if scratch.startswith('QualityCheckedPSTH'):
                n_stimuli.append(combined_nwb.scratch[scratch][:].shape[0])
        assert all(element == n_stimuli[0] for element in n_stimuli) == True, 'Number of Stimuli are not consistent over the PSTH!'
        n_stimuli = n_stimuli[0]
        
        # ------------------------------------------------------------------------------ 
        # Load combined_train nwb file.
        # ------------------------------------------------------------------------------ 
        io_train = NWBHDF5IO(os.path.join(experiment_path, combined_nwb_train[0]), "a") 
        combined_nwb_train = io_train.read()
        try: 
            combined_nwb_train.stimulus_template['StimulusSetTrain']
            print('Simulus Set already exists in train.')
            #display(combined_nwb_train)
            train = True
        except: pass

        n_stimuli_train = []
        for scratch in list(combined_nwb_train.scratch):
            if scratch.startswith('QualityCheckedPSTH'):
                n_stimuli_train.append(combined_nwb_train.scratch[scratch][:].shape[0])
        assert all(element == n_stimuli_train[0] for element in n_stimuli_train) == True, 'Number of Stimuli are not consistent over the PSTH!'
        n_stimuli_train = n_stimuli_train[0]


        # ------------------------------------------------------------------------------ 
        # Load combined_test nwb file.
        # ------------------------------------------------------------------------------ 
        io_test = NWBHDF5IO(os.path.join(experiment_path, combined_nwb_test[0]), "a") 
        combined_nwb_test = io_test.read()
        try: 
            combined_nwb_test.stimulus_template['StimulusSetTest']
            print('Simulus Set already exists in test.')
            test = True
        except: pass

        n_stimuli_test = []
        for scratch in list(combined_nwb_test.scratch):
            if scratch.startswith('QualityCheckedPSTH'):
                n_stimuli_test.append(combined_nwb_test.scratch[scratch][:].shape[0])
        assert all(element == n_stimuli_test[0] for element in n_stimuli_test) == True, 'Number of Stimuli are not consistent over the PSTH!'
        n_stimuli_test = n_stimuli_test[0]
        # print(n_stimuli, n_stimuli_train, n_stimuli_test)

        # ------------------------------------------------------------------------------ 
        # Create StimulusSets.
        # ------------------------------------------------------------------------------ 
        if combined == False or train == False or test == False:
            # for temp, image in enumerate(list_images_sorted):
            #     if count_1 ==1: temp = temp+1

            #     expected_image_name = f'mv_{temp}'
            #     if check_image_order:
            #         if image.split(".")[0] != expected_image_name and image.split(".")[0] !=f'{temp}' and image.split(".")[0] !=f'mv{temp}' and image.split(".")[0] !=f'vid{temp}':
            #                 print(f'Image names do not increase with +1!!! Expected: {expected_image_name} or image{temp} or {temp}, Found: {image.split(".")[0]}')

            external_file = [os.path.join('../VideoStimulusSet', movie_path) for movie_path in list_images_sorted]
            # moviefiles_path = [os.path.join(*experiment_path.split('/')[:-1], 'VideoStimulusSet', movie) for movie in list_images_sorted]
            # combined_nwb_path = os.path.join(experiment_path, [x for x in os.listdir(experiment_path) if x.endswith('combined.nwb')][0])
            # print(moviefiles_path, combined_nwb_path)
            # external_file = [os.path.relpath(movie_path, combined_nwb_path) for movie_path in moviefiles_path]
            print(external_file)
            # ------------------------------------------------------------------------------ 
            # StimulusSet for combined.
            # ------------------------------------------------------------------------------ 
            StimulusMovieFile_combined = ImageSeries(
                name="StimulusSet",
                description=f"This list references external files linking to movies in the stimulus set. \
                    Each movie has an associated 'starting_frame' field, repurposed to serve as the \
                    stimulusID in PSTHs. Filenames with sequential numbers starting from 0 align with these \
                    StimulusIDs. Note: The original function of 'starting_frame' is altered for this specific use case.",
                unit="n.a.",
                external_file=external_file,
                format="external", 
                rate=0.0, 
                starting_frame = np.linspace(0, n_stimuli-1, n_stimuli))
            
            # ------------------------------------------------------------------------------ 
            # StimulusSet for train.
            # ------------------------------------------------------------------------------ 
            StimulusMovieFile_train = ImageSeries(
                name="StimulusSetTrain",
                description="This list references external files linking to movies in the stimulus set. \
                    Each movie has an associated 'starting_frame' field, repurposed to serve as the \
                    stimulusID in PSTHs. Filenames with sequential numbers starting from 0 align with these \
                    StimulusIDs. Note: The original function of 'starting_frame' is altered for this specific use case.",
                unit="n.a.",
                external_file=external_file[:n_stimuli_train],
                format="external", 
                rate=0.0, 
                starting_frame = np.linspace(0, n_stimuli_train-1, n_stimuli_train))
            
            # ------------------------------------------------------------------------------ 
            # StimulusSet for test.
            # ------------------------------------------------------------------------------ 
            StimulusMovieFile_test = ImageSeries(
                name="StimulusSetTest",
                description="This list references external files linking to movies in the stimulus set. \
                    Each movie has an associated 'starting_frame' field, repurposed to serve as the \
                    stimulusID in PSTHs. Filenames with sequential numbers starting from 0 align with these \
                    StimulusIDs. Note: The original function of 'starting_frame' is altered for this specific use case.",
                unit="n.a.",
                external_file=external_file[n_stimuli_train:],
                format="external", 
                rate=0.0, 
                starting_frame = np.linspace(n_stimuli_train, n_stimuli-1, n_stimuli_test))
                
            assert n_stimuli == len(external_file), 'Number of Stimuli does not match number of Images!'

        # ------------------------------------------------------------------------------ 
        # Append to nwb files.
        # ------------------------------------------------------------------------------ 
        text=['']
        if combined == False: 
            try: 
                combined_nwb.add_stimulus_template(timeseries=StimulusMovieFile_combined, use_sweep_table=False) 
                print(f"Added StimulusSet to combined nwb.")
                # display(combined_nwb)
                io.write(combined_nwb)
                io.close()  
                text.append('StimulusSet added to combined nwb.')
            except Exception as error: 
                print("An error occurred:", error) 
                io.close()  
                text.append(error)

        if train == False:
            try: 
                combined_nwb_train.add_stimulus_template(timeseries=StimulusMovieFile_train, use_sweep_table=False)  
                print(f"Added StimulusSet to train nwb.")
                # display(combined_nwb_train)
                io_train.write(combined_nwb_train)
                io_train.close()    
                text.append('StimulusSet added to train nwb.')
            except Exception as error: 
                print("An error occurred:", error) 
                io_train.close() 
                text.append(error)

        if test == False:
            try: 
                combined_nwb_test.add_stimulus_template(timeseries=StimulusMovieFile_test, use_sweep_table=False) 
                print(f"Added StimulusSet to test nwb.")
                # display(combined_nwb_test)
                io_test.write(combined_nwb_test)
                io_test.close()  
                text.append('StimulusSet added to test nwb.')
            except Exception as error: 
                print("An error occurred:", error)    
                io_test.close() 
                text.append(error)

        # ------------------------------------------------------------------------------ 
        # update excel.
        # ------------------------------------------------------------------------------ 
        if text==['']: text = 'StimulusSet created and added to nwb files.'
        update_sheet(df, experiment_path, text)
        

### Step 2: Create per experiment stimulus set from data available in /braintree/data2/active/users/sgouldin/experiments-codebase and add them to the per experiment nwb files.

In [26]:
# ------------------------------------------------------------------------------ 
# Load excel file and find all experiment names goind on brainscore. 
# ------------------------------------------------------------------------------ 
list_of_bs_exp_names = []
df['StimulusSet'] = ''
df['StimulusSetPath'] = ''
for index, row in df.iterrows():
    if row['BrainScore']=='Y': list_of_bs_exp_names.append(row['ImageSet'])
    
experiment_file_paths = glob.glob(os.path.join(root_dir, '[exp]*', '*'))
stimulus_dir          = '/braintree/data2/active/users/sgouldin/experiments-codebase'
stimuli_names         = os.listdir(stimulus_dir)

for experiment_path in experiment_file_paths: 
    experiment_name =  "_".join(os.path.basename(experiment_path).split('.')[0].split('_')[1:])

    print('________________________________________________________________________________')
    print(experiment_name)
    if not experiment_name.startswith('oasis900scrambled'): continue
    
    if experiment_name not in list_of_bs_exp_names: 
        continue 

    
    # ------------------------------------------------------------------------------ 
    # Find Stimulus Directory name for each experiment. 
    # ------------------------------------------------------------------------------ 
    if experiment_name == 'domain-transfer-2023':
        stim_name = [x for x in stimuli_names if x.endswith('domain_transfer') and not x.startswith('.')]
    elif experiment_name == 'HVM-var6-2023':
        stim_name = [x for x in stimuli_names if x.endswith('HVM_var6') and not x.startswith('.')]
    elif experiment_name.startswith('gratingsAdap_'):
        stim_name = [x for x in stimuli_names if x.endswith('gratingsAdap') and not x.startswith('.')]
    elif experiment_name.startswith('gestalt'):
        stim_name = [x for x in stimuli_names if x.endswith('Gestalt') and not x.startswith('.')]
    elif experiment_name.startswith('object_relations'):
        stim_name = [x for x in stimuli_names if x.endswith('ObjectRelationships') and not x.startswith('.')]
    elif experiment_name.startswith('1_shapes'):
        stim_name = [x for x in stimuli_names if x.endswith('shapes') and not x.startswith('.')]
    elif experiment_name.startswith('food'):
        stim_name = [x for x in stimuli_names if x.endswith('Food') and not x.startswith('.')]
    elif experiment_name.startswith('shapenet360'):
        stim_name = [x for x in stimuli_names if x.endswith('ShapeNet360') and not x.startswith('.')]
    elif experiment_name.startswith('shapegen'):
        stim_name = [x for x in stimuli_names if x.endswith('ShapeGens') and not x.startswith('.')]
    elif experiment_name.startswith('sine_wave'):
        stim_name = [x for x in stimuli_names if x.endswith('sinewave_fullfield') and not x.startswith('.')]
    elif experiment_name.startswith('square_sinewave'):
        stim_name = [x for x in stimuli_names if x.endswith('squarewave_fullfield') and not x.startswith('.')]
    elif experiment_name.endswith('oasis900'):
        stim_name = [x for x in stimuli_names if x.endswith('Oasis900') and not x.startswith('.')]
    elif experiment_name.startswith('oasis900_200'):
        stim_name = [x for x in stimuli_names if x.endswith('Oasis900') and not x.startswith('.')]
    elif experiment_name.startswith('oasis100'):
        stim_name = [x for x in stimuli_names if x.endswith('OASIS100_control') and not x.startswith('.')]
    elif experiment_name.startswith('oasis900rotated'):
        stim_name = [x for x in stimuli_names if x.endswith('OasisRotated') and not x.startswith('.')]
    elif experiment_name.startswith('oasis900scrambled'):
        stim_name = [x for x in stimuli_names if x.endswith('OasisScramble') and not x.startswith('.')]
    elif experiment_name.startswith('IAPS-200on'):
        stim_name = [x for x in stimuli_names if x.endswith('IAPS') and not x.startswith('.')]
    elif experiment_name.startswith('flicker'):
        stim_name = [x for x in stimuli_names if x.startswith('flicker') and not x.startswith('.')]
    elif experiment_name.startswith('ko_context_size'):
        stim_name = [f'{x}/ko_context_size' for x in stimuli_names if x.startswith('old_rig1') and not x.startswith('.')]
    elif experiment_name.startswith('muri'):
        stim_name = [f'{x}/RSVP-MURI1320' for x in stimuli_names if x.startswith('old_rig1') and not x.startswith('.')]
    else:
        stim_name = [x for x in stimuli_names if x.endswith(experiment_name) and not x.startswith('.')]

    if len(stim_name)==0: 
        print(f'    No Stim found: {experiment_name}') 
        update_sheet(df, experiment_path, 'No StimulusSet found.')
        continue

    files_starting_with_vid = [file for file in os.listdir(os.path.join(stimulus_dir, stim_name[0])) if file.startswith('vid')]
    ImageStimSetPath = None
    VideoStimSetPath = None

    # ------------------------------------------------------------------------------ 
    # Find 'images' folder in Stimulus Directory. (standard case)
    # ------------------------------------------------------------------------------ 
    if 'images' in os.listdir(os.path.join(stimulus_dir, stim_name[0])): 
        ImageStimSetPath = os.path.join(stimulus_dir, stim_name[0], 'images')
    
    # ------------------------------------------------------------------------------ 
    # Find 'vid...' folder.
    # ------------------------------------------------------------------------------ 
    elif len(files_starting_with_vid) > 0: 
        VideoStimSetPath = os.path.join(stimulus_dir, stim_name[0], files_starting_with_vid[0])

    # ------------------------------------------------------------------------------ 
    # Manually find folders for gratingsAdap.
    # ------------------------------------------------------------------------------ 
    elif experiment_name.startswith('gratingsAdap_'):
        season = experiment_name.split('_')[-1][-1]
        file = f'season{season}'
        list_videos = os.listdir(os.path.join(stimulus_dir, stim_name[0], file))
        if list_videos[0].startswith('mv'): VideoStimSetPath = os.path.join(stimulus_dir, stim_name[0], file)
        if list_videos[0].startswith('im'): ImageStimSetPath = os.path.join(stimulus_dir, stim_name[0], file)
    
    # ------------------------------------------------------------------------------ 
    # Manually find folders for object_relations.
    # ------------------------------------------------------------------------------ 
    elif experiment_name.startswith('object_relations'):
        files_starting_with_vid = [file for file in os.listdir(os.path.join(stimulus_dir, stim_name[0])) if file.startswith('mworks')]
        VideoStimSetPath = os.path.join(stimulus_dir, stim_name[0], files_starting_with_vid[0])


    # ------------------------------------------------------------------------------ 
    # Manually find folders for oasis900.
    # ------------------------------------------------------------------------------ 
    elif experiment_name =='oasis900' or experiment_name =='oasis900_200on':
            ImageStimSetPath = os.path.join(stimulus_dir, stim_name[0], 'image_dicarlo_oasis900')

    # ------------------------------------------------------------------------------ 
    # Manually find folders for oasis100.
    # ------------------------------------------------------------------------------ 
    elif experiment_name.startswith('oasis100'): 
        if experiment_name.endswith('c'): ImageStimSetPath = os.path.join(stimulus_dir, stim_name[0], 'images_control')
        if experiment_name.endswith('o'): ImageStimSetPath = os.path.join(stimulus_dir, stim_name[0], 'images_original')

    # ------------------------------------------------------------------------------ 
    # Manually find folders for square_sinewave.
    # ------------------------------------------------------------------------------ 
    elif experiment_name == 'square_sinewave': 
        VideoStimSetPath = os.path.join(stimulus_dir, stim_name[0], 'squarewave_movies')

    else: print(f'  No Images or Videos found for {experiment_name} {stim_name}')
    # ------------------------------------------------------------------------------ 
    # For the following experiments, either SimulusSet is not found or the nwb files
    # are not creted yet. Once both are done, check how the Simulus Direcories look 
    # and update this part (or the finding StimulusDirecory part.)
    # ------------------------------------------------------------------------------ 

    if experiment_name == 'NSD-COCO': 
        print("To Do")
        continue
    elif experiment_name == 'RF': 
        print("To Do")
        continue
    elif experiment_name == 'flicker': 
        print("To Do")
        continue
    elif experiment_name == 'gestalt': 
        print("To Do")
        continue
    elif experiment_name.startswith('monkeyvalence'): 
        print("To Do")
        continue
    elif experiment_name == 'object_relations': 
        print("To Do")
        continue
    elif experiment_name == 'sine_wave': 
        print("To Do")
        continue

    # ------------------------------------------------------------------------------ 
    # Create StimulusSet for each ImageStimulusSet
    # ------------------------------------------------------------------------------ 
    df_index = np.where(df['ImageSet'].to_numpy() == experiment_name)[0][0]

    if ImageStimSetPath != None: 

        # for file in os.listdir(experiment_path):
        #     if file.endswith('.nwb'): os.remove(os.path.join(experiment_path, file))
            
        check_image_order = True
        count_1 = 0

        if experiment_name =='oasis900' or experiment_name =='oasis900_200on':
            def extract_integer(image_name):
                try:
                    return int(image_name[2:-4])
                except: pass
                
            list_images = os.listdir(ImageStimSetPath)
            list_images_sorted = [x for x in sorted(list_images, key = extract_number) if not x.startswith('.')]
            csv_path = os.path.join('/', *ImageStimSetPath.split('/')[:-1], 'image_dicarlo_oasis900.csv')
            df_csv = pd.read_csv(csv_path)
            mapping = {extract_integer(row['image_file_name']): row['filename'] for index, row in df_csv.iterrows()}
            reverse_mapping = {v: k for k, v in mapping.items()}
            list_images_sorted = sorted(list_images_sorted, key=lambda x: reverse_mapping.get(x, float('inf')))
            check_image_order = False

        elif experiment_name =='oasis900rotated':
            """
            According to the mwel file, the stimulus is organized as first 900 ori and then 900 rot.
            file_path = os.path.join('/', *path.split('/')[:-1],'image_set_definition_oriandrotated.mwel' ) 
            with open(file_path, 'r') as file:
                mwel_content = file.read()
            print(mwel_content)
            """
            list_images = os.listdir(ImageStimSetPath)
            list_images_sorted = [x for x in sorted(list_images, key = extract_number) if not x.startswith('.')]
            list_images_sorted_ori = [x for x in list_images_sorted if x.startswith('im_ori')]
            list_images_sorted_rot = [x for x in list_images_sorted if x.startswith('im_rot')]
            list_images_sorted = list_images_sorted_ori + list_images_sorted_rot
            check_image_order = False

        elif experiment_name == 'oasis900scrambled' or experiment_name == 'oasis900scrambled_200on': 
            filename = 'image_set_definition_oriandscramble.mwel'
            list_images_sorted = []
            with open(os.path.join(stimulus_dir, stim_name[0], filename), 'r') as file:
                mwel_content = file.read()
            for substring in mwel_content.split('var imagefiles')[-1].split('",\n"'):
                list_images_sorted.append(substring.split('/')[-1])
            check_image_order = False

        else:
            list_images = os.listdir(ImageStimSetPath)
            list_images_sorted = [x for x in sorted(list_images, key = extract_number) if not x.startswith('.')]

        if experiment_name == 'shapenet360' or experiment_name =='1_shapes' or experiment_name =='food' or experiment_name =='shapegen_static': # Add when list_images_sorted start with 1 and not 0 in filename. 
            count_1 = 1
            
        print(experiment_name, ImageStimSetPath, list_images_sorted)
        df.at[df_index, "StimulusSetPath"] = ImageStimSetPath
        update_exp_nwb(experiment_path, list_images_sorted, ImageStimSetPath, count_1, check_image_order)

    # ------------------------------------------------------------------------------ 
    # Create StimulusSet for each VideoStimulusSet
    # ------------------------------------------------------------------------------ 
    if VideoStimSetPath != None: 
        print(experiment_name, VideoStimSetPath)
        df.at[df_index, "StimulusSetPath"] = VideoStimSetPath
        list_movies = os.listdir(VideoStimSetPath)
        list_movies_sorted = [x for x in sorted(list_movies, key = extract_number) if not x.startswith('.')]

        if experiment_name == 'square_sinewave':
            file_path = '/braintree/data2/active/users/sgouldin/experiments-codebase/squarewave_fullfield/movie_definition_squarewave_set1.mwel'
            list_movies_sorted = []
            with open(file_path, 'r') as file:
                mwel_content = file.read()
            for substring in mwel_content.split('var imagefiles')[-1].split('",\n"'):
                try: 
                    subsubstring = substring.split('"')
                    for sub in subsubstring:
                        if sub.startswith('squarewave_movies'):
                            list_movies_sorted.append(sub.split('/')[-1])
                except: list_movies_sorted.append(substring.split('/')[-1])


        # ------------------------------------------------------------------------------ 
        # Copy movies into experiment file.
        # ------------------------------------------------------------------------------ 
        image_set_path = '/'.join(experiment_path.split('/')[:-1])
        try: os.mkdir(os.path.join(image_set_path, 'VideoStimulusSet'))
        except: pass
        rename_flag = False
        for movie in list_movies_sorted:
            try: shutil.copy2(os.path.join(VideoStimSetPath, movie), os.path.join(image_set_path, 'VideoStimulusSet'))
            except: rename_flag = True
        
        # ------------------------------------------------------------------------------ 
        # Update nwb files.
        # ------------------------------------------------------------------------------ 
        list_movies_sorted_new = []
        if rename_flag == False:
            if experiment_name == 'motionset1' or experiment_name == 'moca' or experiment_name == 'afv'  or experiment_name == 'faceemovids' or experiment_name.startswith('gratingsAdap_'):
                for movie in list_movies_sorted:
                    match = re.search(r'\d+', movie)
                    if match:
                        number = int(match.group())
                        movie_filename_new = f'exp_{experiment_name}_{number}.mp4'
                        list_movies_sorted_new.append(movie_filename_new)
                        moviepath = os.path.join(image_set_path, 'VideoStimulusSet')
                        os.rename(os.path.join(moviepath, movie), os.path.join(moviepath, movie_filename_new))

                # update_exp_nwb_movies(experiment_path, list_movies_sorted, 0, check_image_order=True)
            elif experiment_name == 'gestalt' or experiment_name == 'object_relations' or experiment_name == 'Co3D':
                for movie in list_movies_sorted:
                    match = re.search(r'\d+', movie)
                    if match:
                        number = int(match.group())
                        movie_filename_new = f'exp_{experiment_name}_{number-1}.mp4'
                        list_movies_sorted_new.append(movie_filename_new)
                        moviepath = os.path.join(image_set_path, 'VideoStimulusSet')
                        os.rename(os.path.join(moviepath, movie), os.path.join(moviepath, movie_filename_new))
                # update_exp_nwb_movies(experiment_path, list_movies_sorted, 1, check_image_order=True)
            elif experiment_name == 'square_sinewave':
                for movie, number in zip(list_movies_sorted, range(len(list_movies_sorted))):
                    movie_filename_new = f'exp_{experiment_name}_{number}.mp4'
                    list_movies_sorted_new.append(movie_filename_new)
                    moviepath = os.path.join(image_set_path, 'VideoStimulusSet')
                    os.rename(os.path.join(moviepath, movie), os.path.join(moviepath, movie_filename_new))


        else: list_movies_sorted_new = list_movies_sorted
        
        update_exp_nwb_movies(experiment_path, list_movies_sorted_new, 0, check_image_order=True)

 

________________________________________________________________________________
muri1320
________________________________________________________________________________
ko_context_size
________________________________________________________________________________
robustness_guy_d1_v40
________________________________________________________________________________
muri1320-2023-v1
________________________________________________________________________________
HVM-var6-subset-2023
________________________________________________________________________________
images_in_context
________________________________________________________________________________
muri1320-2023-v0
________________________________________________________________________________
domain-transfer-2023
________________________________________________________________________________
HVM-var6-2023
________________________________________________________________________________
facesMSFDE
________________________

root pynwb.file.NWBFile at 0x140563098577808
Fields:
  devices: {
    DET36A2 Biased Si Detector <class 'pynwb.device.Device'>,
    Electrode <class 'pynwb.device.Device'>,
    LG UltraGear <class 'pynwb.device.Device'>,
    RecordingSystem <class 'pynwb.device.Device'>,
    Ripple Neuro <class 'pynwb.device.Device'>,
    Software Used <class 'pynwb.device.Device'>
  }
  electrode_groups: {
    group_A <class 'pynwb.ecephys.ElectrodeGroup'>,
    group_B <class 'pynwb.ecephys.ElectrodeGroup'>,
    group_C <class 'pynwb.ecephys.ElectrodeGroup'>,
    group_D <class 'pynwb.ecephys.ElectrodeGroup'>,
    group_E <class 'pynwb.ecephys.ElectrodeGroup'>,
    group_F <class 'pynwb.ecephys.ElectrodeGroup'>
  }
  electrodes: electrodes <class 'hdmf.common.table.DynamicTable'>
  experiment_description: Task: Rapid serial visual presentation (RSVP).
  experimenter: ['Goulding, Sarah']
  file_create_date: [datetime.datetime(2023, 12, 6, 16, 19, 37, 275556, tzinfo=tzoffset(None, -18000))]
  identifier: c3faa8b5-86cd-499d-ba83-056e6e7e954f
  institution: McGovern Institute for Brain Research
  keywords: <StrDataset for HDF5 dataset "keywords": shape (4,), type "|O">
  lab: DiCarlo
  scratch: {
    CombinedQualityCheckedPSTHs <class 'pynwb.core.ScratchData'>,
    QualityCheckedPSTH_20230515_135050 <class 'pynwb.core.ScratchData'>,
    QualityCheckedPSTH_20230516_121545 <class 'pynwb.core.ScratchData'>,
    QualityCheckedPSTH_20230517_121232 <class 'pynwb.core.ScratchData'>,
    QualityCheckedPSTH_20230802_134310 <class 'pynwb.core.ScratchData'>,
    QualityCheckedPSTH_20230803_115110 <class 'pynwb.core.ScratchData'>,
    QualityElectrodesMasks <class 'pynwb.core.ScratchData'>
  }
  session_description: Integrated NWB file for the experiment, featuring PSTH data verified for quality from each session.
  session_id: exp_oasis900scrambled
  session_start_time: 2023-05-17 12:12:32-04:00
  stimulus_template: {
    StimulusSet <class 'pynwb.base.Images'>
  }
  subject: subject pynwb.file.Subject at 0x140563087945744
Fields:
  age__reference: birth
  date_of_birth: 2014-06-22 00:00:00-04:00
  description: monkey
  sex: M
  species: Macaca mulatta
  subject_id: pico

  surgery: 3x Utah Array Implant + Headpost
  timestamps_reference_time: 2023-05-17 12:12:32-04:00

Simulus Set already exists in train.
Simulus Set already exists in test.
________________________________________________________________________________
oasis900rotated
________________________________________________________________________________
oasis900_200on
________________________________________________________________________________
monkeyvalence2-200on
________________________________________________________________________________
monkeyvalence3
________________________________________________________________________________
monkeyvalence4
________________________________________________________________________________
monkeyvalence6
________________________________________________________________________________
monkeyvalence7
________________________________________________________________________________
monkeyvalence8
________________________________________________________________________________
oasis900scrambled_200on
oasis900scrambled_200on /braintree/data2/active

root pynwb.file.NWBFile at 0x140563048629904
Fields:
  devices: {
    DET36A2 Biased Si Detector <class 'pynwb.device.Device'>,
    Electrode <class 'pynwb.device.Device'>,
    LG UltraGear <class 'pynwb.device.Device'>,
    RecordingSystem <class 'pynwb.device.Device'>,
    Ripple Neuro <class 'pynwb.device.Device'>,
    Software Used <class 'pynwb.device.Device'>
  }
  electrode_groups: {
    group_A <class 'pynwb.ecephys.ElectrodeGroup'>,
    group_B <class 'pynwb.ecephys.ElectrodeGroup'>,
    group_C <class 'pynwb.ecephys.ElectrodeGroup'>,
    group_D <class 'pynwb.ecephys.ElectrodeGroup'>,
    group_E <class 'pynwb.ecephys.ElectrodeGroup'>,
    group_F <class 'pynwb.ecephys.ElectrodeGroup'>
  }
  electrodes: electrodes <class 'hdmf.common.table.DynamicTable'>
  experiment_description: Task: Rapid serial visual presentation (RSVP).
  experimenter: ['Goulding, Sarah']
  file_create_date: [datetime.datetime(2023, 12, 6, 16, 21, 40, 553740, tzinfo=tzoffset(None, -18000))]
  identifier: f72981b9-54e2-49f9-9888-6ad1d70beb35
  institution: McGovern Institute for Brain Research
  keywords: <StrDataset for HDF5 dataset "keywords": shape (4,), type "|O">
  lab: DiCarlo
  scratch: {
    CombinedQualityCheckedPSTHs <class 'pynwb.core.ScratchData'>,
    QualityCheckedPSTH_20230821_121635 <class 'pynwb.core.ScratchData'>,
    QualityCheckedPSTH_20230822_155903 <class 'pynwb.core.ScratchData'>,
    QualityElectrodesMasks <class 'pynwb.core.ScratchData'>
  }
  session_description: Integrated NWB file for the experiment, featuring PSTH data verified for quality from each session.
  session_id: exp_oasis900scrambled_200on
  session_start_time: 2023-08-21 12:16:35-04:00
  stimulus_template: {
    StimulusSet <class 'pynwb.base.Images'>
  }
  subject: subject pynwb.file.Subject at 0x140563086957136
Fields:
  age__reference: birth
  date_of_birth: 2014-06-22 00:00:00-04:00
  description: monkey
  sex: M
  species: Macaca mulatta
  subject_id: pico

  surgery: 3x Utah Array Implant + Headpost
  timestamps_reference_time: 2023-08-21 12:16:35-04:00

Simulus Set already exists in train.
Simulus Set already exists in test.


In [4]:
# ------------------------------------------------------------------------------ 
# Update Sheet 2 of Excel File.
# ------------------------------------------------------------------------------ 
xls = pd.ExcelFile(f'{os.path.dirname(cwd)}/pico_inventory.xlsx')
sheets = {sheet: xls.parse(sheet) for sheet in xls.sheet_names}

sheets['Sheet2'] = df  

with pd.ExcelWriter(f'{os.path.dirname(cwd)}/pico_inventory.xlsx', engine='openpyxl', mode='w') as writer:
    for sheet_name, sheet_df in sheets.items():
        sheet_df.to_excel(writer, sheet_name=sheet_name, index=False)  

### Step 3: Move Files to dandi_folder 

In [5]:
# ------------------------------------------------------------------------------ 
# Create a Train and Test DandiSet
# ------------------------------------------------------------------------------ 
destination_test  = '/braintree/home/aliya277/dandi_folder_test'
destination_train = '/braintree/home/aliya277/dandi_folder_train'
try: 
    os.mkdir(destination_test)
    os.mkdir(destination_train)
except: pass

# big experiment and video experiment
chosen_exp  = ['HVM-var6-2023', 'faceemovids' ]

for experiment in chosen_exp:
    # ------------------------------------------------------------------------------ 
    # Make Direcotries.
    # ------------------------------------------------------------------------------ 
    try: 
        os.mkdir(os.path.join(destination_test, experiment))
        os.mkdir(os.path.join(destination_train, experiment))
    except Exception as error: print(error)
    try: 
        os.mkdir(os.path.join(destination_test, experiment , f'{experiment}.sub_pico'))
        os.mkdir(os.path.join(destination_train, experiment, f'{experiment}.sub_pico'))
    except Exception as error: print(error)

    experiment_path = os.path.join(root_dir, f'exp_{experiment}')
    folders = os.listdir(experiment_path)
    for folder in folders:
        if folder.startswith('exp_'):
            # ------------------------------------------------------------------------------ 
            # Copy train and test nwb files.
            # ------------------------------------------------------------------------------ 
            for file in os.listdir(os.path.join(experiment_path, folder)):
                if file.endswith('test.nwb'): 
                    try:
                        shutil.copy2(os.path.join(experiment_path, folder, file), os.path.join(destination_test, experiment, f'{experiment}.sub_pico'))
                    except: pass
                    filename_test = file
                if file.endswith('train.nwb'): 
                    try:
                        shutil.copy2(os.path.join(experiment_path, folder, file), os.path.join(destination_train, experiment, f'{experiment}.sub_pico'))
                    except: pass
                    filename_train = file
        elif folder.startswith('Video'):
            # ------------------------------------------------------------------------------ 
            # Copy VideoStimulusSets.
            # ------------------------------------------------------------------------------
            try: 
                shutil.copytree(os.path.join(experiment_path, folder), os.path.join(destination_test, experiment, folder))
                shutil.copytree(os.path.join(experiment_path, folder), os.path.join(destination_train, experiment, folder))
            except Exception as error: print(error)




### Step 4: Upload Folders following steps on this website: https://www.dandiarchive.org/handbook/13_upload/

Run the following commands in commandline (change train to test, for testset): 

nwbinspector /braintree/home/aliya277/dandi_folder_train --config dandi
dandi download https://dandiarchive.org/dandiset/000720/draft
cd 000720
dandi organize /braintree/home/aliya277/dandi_folder_train
nwbinspector /braintree/home/aliya277/000720 --config dandi
dandi upload

to download: 
dandi download DANDI:000720

In [8]:
# ------------------------------------------------------------------------------ 
# Display files in DandiSet folder. 
# ------------------------------------------------------------------------------ 
path = '/braintree/home/aliya277/000720/sub-pico'
def display_nwb(file):
    io = NWBHDF5IO(file, "r") 
    combined_nwb = io.read()
    display(combined_nwb)
    print(combined_nwb.scratch)#['QualityCheckedPSTH_20230908_100255']['psth'][:])
    for key in combined_nwb.scratch.keys():
        if key.startswith('QualityCheckedPSTH'): 
            # psth = nwbfile.scratch[key][:]
            print(key)
            print(combined_nwb.scratch[key][:])

    # display(combined_nwb.stimulus_template['StimulusSetTrain'])
    # try: display(combined_nwb.stimulus_template['StimulusSetTrain'].external_file[:])
    # except: pass
    # from nwbwidgets import nwb2widget
    # nwb2widget(combined_nwb)
    io.close()
for file in os.listdir(path):
    display_nwb(os.path.join(path, file))

root pynwb.file.NWBFile at 0x139905791675792
Fields:
  devices: {
    DET36A2 Biased Si Detector <class 'pynwb.device.Device'>,
    Electrode <class 'pynwb.device.Device'>,
    LG UltraGear <class 'pynwb.device.Device'>,
    RecordingSystem <class 'pynwb.device.Device'>,
    Ripple Neuro <class 'pynwb.device.Device'>,
    Software Used <class 'pynwb.device.Device'>
  }
  electrode_groups: {
    group_A <class 'pynwb.ecephys.ElectrodeGroup'>,
    group_B <class 'pynwb.ecephys.ElectrodeGroup'>,
    group_C <class 'pynwb.ecephys.ElectrodeGroup'>,
    group_D <class 'pynwb.ecephys.ElectrodeGroup'>,
    group_E <class 'pynwb.ecephys.ElectrodeGroup'>,
    group_F <class 'pynwb.ecephys.ElectrodeGroup'>
  }
  electrodes: electrodes <class 'hdmf.common.table.DynamicTable'>
  experiment_description: Task: Rapid serial visual presentation (RSVP).
  experimenter: ['Goulding, Sarah']
  file_create_date: [datetime.datetime(2023, 12, 4, 11, 46, 56, 485919, tzinfo=tzoffset(None, -18000))]
  identifier: b3e5ae22-d000-48b6-a0b9-2a4e79801c90
  institution: McGovern Institute for Brain Research
  keywords: <StrDataset for HDF5 dataset "keywords": shape (4,), type "|O">
  lab: DiCarlo
  scratch: {
    QualityCheckedPSTH_20230214_154007 <class 'pynwb.core.ScratchData'>,
    QualityCheckedPSTH_20230215_150717 <class 'pynwb.core.ScratchData'>,
    QualityCheckedPSTH_20230216_163217 <class 'pynwb.core.ScratchData'>,
    QualityCheckedPSTH_20230217_142420 <class 'pynwb.core.ScratchData'>,
    QualityCheckedPSTH_20230221_142542 <class 'pynwb.core.ScratchData'>,
    QualityCheckedPSTH_20230222_145258 <class 'pynwb.core.ScratchData'>,
    QualityCheckedPSTH_20230223_150327 <class 'pynwb.core.ScratchData'>,
    QualityCheckedPSTH_20230224_143600 <class 'pynwb.core.ScratchData'>,
    QualityCheckedPSTH_20230227_151407 <class 'pynwb.core.ScratchData'>,
    QualityCheckedPSTH_20230301_142202 <class 'pynwb.core.ScratchData'>,
    QualityCheckedPSTH_20230302_151005 <class 'pynwb.core.ScratchData'>,
    QualityCheckedPSTH_20230303_154230 <class 'pynwb.core.ScratchData'>,
    QualityElectrodesMasks <class 'pynwb.core.ScratchData'>,
    TrainStimuliIDs <class 'pynwb.core.ScratchData'>
  }
  session_description: Integrated NWB file for the experiment, featuring PSTH data verified for quality from each session.
  session_id: exp_HVM-var6-2023
  session_start_time: 2023-02-16 16:32:17-05:00
  stimulus_template: {
    StimulusSetTrain <class 'pynwb.base.Images'>
  }
  subject: subject pynwb.file.Subject at 0x139905665985680
Fields:
  age__reference: birth
  date_of_birth: 2014-06-22 00:00:00-04:00
  description: monkey
  sex: M
  species: Macaca mulatta
  subject_id: pico

  surgery: 3x Utah Array Implant + Headpost
  timestamps_reference_time: 2023-02-16 16:32:17-05:00

{'QualityCheckedPSTH_20230214_154007': <pynwb.core.ScratchData object at 0x7f3e59df8bd0>, 'QualityCheckedPSTH_20230215_150717': <pynwb.core.ScratchData object at 0x7f3e59dfa6d0>, 'QualityCheckedPSTH_20230216_163217': <pynwb.core.ScratchData object at 0x7f3e59dfb8d0>, 'QualityCheckedPSTH_20230217_142420': <pynwb.core.ScratchData object at 0x7f3e59df8650>, 'QualityCheckedPSTH_20230221_142542': <pynwb.core.ScratchData object at 0x7f3e59df8890>, 'QualityCheckedPSTH_20230222_145258': <pynwb.core.ScratchData object at 0x7f3e59e12c50>, 'QualityCheckedPSTH_20230223_150327': <pynwb.core.ScratchData object at 0x7f3e59e12b10>, 'QualityCheckedPSTH_20230224_143600': <pynwb.core.ScratchData object at 0x7f3e59e13c90>, 'QualityCheckedPSTH_20230227_151407': <pynwb.core.ScratchData object at 0x7f3e59e12110>, 'QualityCheckedPSTH_20230301_142202': <pynwb.core.ScratchData object at 0x7f3e59e11990>, 'QualityCheckedPSTH_20230302_151005': <pynwb.core.ScratchData object at 0x7f3e59e11ad0>, 'QualityCheckedPSTH_

root pynwb.file.NWBFile at 0x139905771668880
Fields:
  devices: {
    DET36A2 Biased Si Detector <class 'pynwb.device.Device'>,
    Electrode <class 'pynwb.device.Device'>,
    LG UltraGear <class 'pynwb.device.Device'>,
    RecordingSystem <class 'pynwb.device.Device'>,
    Ripple Neuro <class 'pynwb.device.Device'>,
    Software Used <class 'pynwb.device.Device'>
  }
  electrode_groups: {
    group_A <class 'pynwb.ecephys.ElectrodeGroup'>,
    group_B <class 'pynwb.ecephys.ElectrodeGroup'>,
    group_C <class 'pynwb.ecephys.ElectrodeGroup'>,
    group_D <class 'pynwb.ecephys.ElectrodeGroup'>,
    group_E <class 'pynwb.ecephys.ElectrodeGroup'>,
    group_F <class 'pynwb.ecephys.ElectrodeGroup'>
  }
  electrodes: electrodes <class 'hdmf.common.table.DynamicTable'>
  experiment_description: Task: Rapid serial visual presentation (RSVP).
  experimenter: ['Goulding, Sarah']
  file_create_date: [datetime.datetime(2023, 12, 4, 10, 11, 17, 607222, tzinfo=tzoffset(None, -18000))]
  identifier: f946be15-fae6-499e-8a97-c7a94f5bc626
  institution: McGovern Institute for Brain Research
  keywords: <StrDataset for HDF5 dataset "keywords": shape (4,), type "|O">
  lab: DiCarlo
  scratch: {
    QualityCheckedPSTH_20230908_100255 <class 'pynwb.core.ScratchData'>,
    QualityCheckedPSTH_20230908_111918 <class 'pynwb.core.ScratchData'>,
    QualityElectrodesMasks <class 'pynwb.core.ScratchData'>,
    TrainStimuliIDs <class 'pynwb.core.ScratchData'>
  }
  session_description: Integrated NWB file for the experiment, featuring PSTH data verified for quality from each session.
  session_id: exp_faceemovids
  session_start_time: 2023-09-08 10:02:55-04:00
  stimulus_template: {
    StimulusSetTrain <class 'pynwb.image.ImageSeries'>
  }
  subject: subject pynwb.file.Subject at 0x139905672920016
Fields:
  age__reference: birth
  date_of_birth: 2014-06-22 00:00:00-04:00
  description: monkey
  sex: M
  species: Macaca mulatta
  subject_id: pico

  surgery: 3x Utah Array Implant + Headpost
  timestamps_reference_time: 2023-09-08 10:02:55-04:00

{'QualityCheckedPSTH_20230908_100255': <pynwb.core.ScratchData object at 0x7f3e53ef0850>, 'QualityCheckedPSTH_20230908_111918': <pynwb.core.ScratchData object at 0x7f3e53ef0ed0>, 'QualityElectrodesMasks': <pynwb.core.ScratchData object at 0x7f3e53ef0d90>, 'TrainStimuliIDs': <pynwb.core.ScratchData object at 0x7f3e53ef2490>}
QualityCheckedPSTH_20230908_100255
[[[[1. 2. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [4. 1. 1. ... 2. 1. 1.]
   ...
   [0. 0. 1. ... 0. 0. 1.]
   [1. 1. 2. ... 0. 0. 0.]
   [1. 0. 0. ... 0. 1. 3.]]

  [[2. 0. 2. ... 0. 0. 1.]
   [0. 1. 3. ... 0. 0. 1.]
   [0. 1. 0. ... 0. 1. 0.]
   ...
   [0. 0. 1. ... 0. 0. 0.]
   [0. 0. 0. ... 1. 2. 3.]
   [4. 0. 0. ... 0. 0. 1.]]

  [[3. 2. 0. ... 0. 0. 3.]
   [1. 5. 4. ... 5. 7. 7.]
   [2. 2. 3. ... 0. 0. 3.]
   ...
   [0. 0. 1. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]]

  [[0. 0. 1. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 1.]
   [0. 0. 2. ... 0. 0. 0.]
   ...
   [0. 0. 1. ... 0. 3. 1.]
   [0. 1. 1

### Step 5: Create Data Catalog Master Excel.

In [14]:
master_excel_file = '/pico_data_catalog.xlsx'
data_catalog = 'DiCarlo Data Catalog.xlsx'


data = pd.read_excel(os.path.join(os.path.dirname(cwd), data_catalog), sheet_name='Pico Proposed Data Sets')
new_header = data.iloc[0]  
data = data[1:]       
data.columns = new_header  
data = data.fillna('empty')  
data_catalog = data

inventory = pd.read_excel( os.path.dirname(cwd)+'/pico_inventory.xlsx' , sheet_name='Sheet2')
inventory['stimulus'] = inventory['stimulus'].replace("['', 'Done. ']", 'Done')
inventory.rename(columns={'stimulus': 'StimSet added'}, inplace=True)

def normalize_string(s):
    return s.lower().replace('_', ' ').replace('-', ' ')

data_catalog['NormalizedColumn1'] = data_catalog['Alias(es)'].apply(normalize_string)
data_catalog['NormalizedColumn2'] = data_catalog['Semantic Name'].apply(normalize_string)

selected_columns_df1 = inventory[['ImageSet', 'Num Rec. Sess.', 'Num Has SpikeTime', 'Num Has psth', 'Num Has Excel', 'BrainScore', 'StimSet added']]
selected_columns_df2 = ['proposed by', 'stimuli type', '# stimuli', '# repetitions', 'Size shown (deg)', 'Duration on/off (ms)']

import pandas as pd

# Assuming df1 and df2 are your original DataFrames
# df1 = ...
# df2 = ...

# Columns to be added from df2
columns_to_add = ['ColumnX', 'ColumnY']  # Replace with actual column names

# First merge: df1['ImageSet'] with df2['ColumnA']
merge_a = pd.merge(selected_columns_df1, data_catalog[selected_columns_df2 + ['NormalizedColumn1']], left_on='ImageSet', right_on='NormalizedColumn1', how='left')

# Second merge: df1['ImageSet'] with df2['ColumnB']
merge_b = pd.merge(selected_columns_df1, data_catalog[selected_columns_df2 + ['NormalizedColumn2']], left_on='ImageSet', right_on='NormalizedColumn2', how='left')

# Combine the results from both merges
combined_results = merge_a.combine_first(merge_b)

# Keep only the new columns and the 'ImageSet' column for joining back to df1
combined_results = combined_results[['ImageSet'] + selected_columns_df2]

# Add the combined results back to df1
df1 = pd.merge(selected_columns_df1, combined_results, on='ImageSet', how='left')

# Now df1 contains the new columns with data from df2 where matches were found

display(df1)

df1.to_excel(os.path.dirname(cwd)+master_excel_file, index=False)



,ImageSet,Num Rec. Sess.,Num Has SpikeTime,Num Has psth,Num Has Excel,BrainScore,StimSet added,proposed by,stimuli type,# stimuli,# repetitions,Size shown (deg),Duration on/off (ms)
0,1_shapes,3,3,3,3,Y,Done,NaN,NaN,NaN,NaN,NaN,NaN
1,Alireza_paradigm1,1,1,1,1,M,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Alireza_paradigm2,1,1,1,1,M,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Co3D,4,4,4,4,Y,Done,NaN,NaN,NaN,NaN,NaN,NaN
4,HVM-var6,2,0,0,1,M,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,shapenet360,3,3,3,3,Y,Done,Yoon,images,1561,30,8,200/100
101,shinecut,1,1,1,1,Y,Done,Ko,images,45,30,8,100/100
102,sine_wave,2,2,2,2,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN
103,square_sinewave,1,1,1,1,Y,Done,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
master_excel_file = 'pico_data_catalog.xlsx'
data_catalog = 'DiCarlo Data Catalog.xlsx'


data = pd.read_excel(os.path.join(os.path.dirname(cwd), data_catalog), sheet_name='Pico Proposed Data Sets')
new_header = data.iloc[0]  
data = data[1:]       
data.columns = new_header  
data = data.fillna('empty')  
data_catalog = data
#data_catalog.rename(columns={'Alias(es)': 'ImageSet'}, inplace=True)

inventory = pd.read_excel( os.path.dirname(cwd)+'/pico_inventory.xlsx' , sheet_name='Sheet2')

def normalize_string(s):
    return s.lower().replace('_', ' ').replace('-', ' ')

# Normalize the 'ImageSet' column in df1
inventory['NormalizedImageSet'] = inventory['ImageSet'].apply(normalize_string)


# Normalize the 'Semantic Name' column in df2
data_catalog['NormalizedColumn1'] = data_catalog['Alias(es)'].apply(normalize_string)
data_catalog['NormalizedColumn2'] = data_catalog['Semantic Name'].apply(normalize_string)

# Create a set for faster lookup
normalized_imageset_set = set(inventory['NormalizedImageSet'])

# Function to check if either column matches with ImageSet
def check_match(row):
    return row['NormalizedColumn1'] in normalized_imageset_set or row['NormalizedColumn2'] in normalized_imageset_set

# Apply the function to each row in df2
data_catalog['Matches'] = data_catalog.apply(check_match, axis=1)

# Filter df2 to only include rows where there is a match
matched_df2 = data_catalog[data_catalog['Matches']]

# Now matched_df2 contains rows from df2 where either Column1 or Column2 matches with ImageSet in df1


display(matched_df2)
display(matched_df2.sort_values(by='NormalizedImageSet'))
display(inventory.sort_values(by='NormalizedImageSet'))


# selected_columns_df1 = inventory[['NormalizedImageSet', 'Num Rec. Sess.', 'Num Has SpikeTime', 'Num Has psth', 'Num Has Excel', 'BrainScore', 'stimulus']]
# selected_columns_df2 = data_catalog[['NormalizedImageSet', 'proposed by', 'stimuli type', '# stimuli', '# repetitions', 'Size shown (deg)', 'Duration on/off (ms)']]

# merged_df = pd.merge(selected_columns_df1, selected_columns_df2, on='NormalizedImageSet')
# merged_df = pd.merge(selected_columns_df1, selected_columns_df2, on='NormalizedImageSet')
# display(merged_df)

# project/image set name, owner, stimuli type (images or videos), # stimuli, total num repetitions (from psth), size shown (deg), Duration on/off (ms), has spike times, has psth, has excel, in dandi set

,Semantic Name,Alias(es),proposed by,stimulus set available,neural data recorded,available in Brain-Score,stimuli source,stimuli type,# stimuli,stimuli obtained by,...,species,Brain Region(s),Size shown (deg),Duration on/off (ms),full stimuli run (days),Notes,NaN,NormalizedColumn1,NormalizedColumn2,Matches
3,MURI1320,MURI1320,Jim,empty,empty,empty,empty,images,1320,empty,...,empty,empty,8,100/100,7,"10 categories, 132 images/category - 8 categor...",empty,muri1320,muri1320,True
4,HVM var6,HVM var6,Tiago/Martin/Marlia,empty,empty,empty,empty,images,2560,empty,...,empty,empty,8,100/100,12,used for training domain_transfer decoders (am...,empty,hvm var6,hvm var6,True
5,dicarlo.Sanghavi2021domain_transfer,domain_transfer,Martin/Tiago/Ko/Marlia,empty,empty,empty,empty,images,3138,empty,...,macaque,empty,8,100/100,empty,early summer - Marlia/Martin/Tiago - interleav...,empty,domain transfer,dicarlo.sanghavi2021domain transfer,True
17,Images_in_context,Images_in_context,Ko,empty,empty,empty,empty,images,246,empty,...,macaque,IT,8,100/100,2023-01-02 00:00:00,Imagesets requested by Ko (pilot data for futu...,empty,images in context,images in context,True
20,Faces_Transformations,facescrub-small,Suleman,empty,empty,empty,empty,images,1248,empty,...,empty,empty,8,100/100,empty,Early Run,empty,facescrub small,faces transformations,True
23,shinecut,empty,Ko,empty,empty,empty,empty,images,45,empty,...,empty,empty,8,100/100,2023-01-02 00:00:00,empty,empty,empty,shinecut,True
26,RF_mapping Yoon,empty,Yoon,empty,empty,empty,empty,images,99,empty,...,empty,empty,empty,1000,2023-01-02 00:00:00,white bars presented at different positions ac...,empty,empty,rf mapping yoon,True
30,objectsize,empty,Ko,empty,empty,empty,empty,images,401,empty,...,empty,empty,8,100/100,2023-01-02 00:00:00,empty,empty,empty,objectsize,True
32,Oasis100o,OASIS100o,Alina/Ko,empty,empty,empty,empty,images,100,empty,...,empty,empty,8,100/100,2023-01-03 00:00:00,empty,empty,oasis100o,oasis100o,True
33,Oasis100c,OASIS100c,Alina/Ko,empty,empty,empty,empty,images,100,empty,...,empty,empty,8,100/100,2023-01-03 00:00:00,empty,empty,oasis100c,oasis100c,True


# extra

In [36]:
def extract_integer(image_name):
    try:
        return int(image_name[2:-4])
    except: pass
list_images = os.listdir(ImageStimSetPath)
list_images_sorted = [x for x in sorted(list_images, key = extract_number) if not x.startswith('.')]    
csv_path = os.path.join('/', *ImageStimSetPath.split('/')[:-1], 'image_dicarlo_oasis900.csv')
df_csv = pd.read_csv(csv_path)
mapping = {extract_integer(row['image_file_name']): row['filename'] for index, row in df_csv.iterrows()}
reverse_mapping = {v: k for k, v in mapping.items()}
list_images_sorted = sorted(list_images_sorted, key=lambda x: reverse_mapping.get(x, float('inf')))
check_image_order = False

print(list_images_sorted)

['877b51675e0bb891d0631024c8c7ea45599db630.jpg', '5e6dab5ce08b223b8913633e98c3525e1463e831.jpg', 'fb9306e586f1b326bfc7939ece43a0ed3764d0ef.jpg', '6c5d8a0d44793f0eb972f5e088c6c35de551080b.jpg', '8ff94dafa817a0aa75b3c1e49bac4fb1f72bfac3.jpg', 'bdc08de7d34ea2620d5a4f08ecd63956f33f3dda.jpg', '24e4c63dbf32f10a6f0c0238685959782e51ec3b.jpg', '3fb2f0001668c1f09352b6f9683d3da6df27dbfc.jpg', 'e7b519b28b372a47e30d91f42cac276d9c5fdf16.jpg', 'b95351a1f1e61e03b899c4841668511b3ef74591.jpg', '0ca9940ddc8a8fe5b27275fc1d418e2b4d9be74c.jpg', 'ef5dee079f36f324ec045a7ecf80f606592c606b.jpg', '64de05a2c75f2d4fc6eeaa558f5791b914a196f5.jpg', '97e8c7b1532691f53be2418b8ba1a9782ab1fe2a.jpg', '1024d27b1151279a866a6639423e9c2c13361d00.jpg', '12bb725a7516012f5fb617e1a3320361a428993f.jpg', '74be5b2810cbb04803af85efd276aee0272cd85d.jpg', '9569850a3b06585779332c50efb18b446da618e6.jpg', '0afe907d2bd760c76d806e85887c689fc72fac54.jpg', '2d9a142d1e90ed196cd4902790bc270f04943f3d.jpg', '51951fdf93eac4fc888987a24cc09c61fff2e1

In [ ]:
list_of_bs_exp_names = []
df = pd.read_excel( os.path.dirname(cwd)+'/pico_inventory.xlsx' , sheet_name='Sheet2')

def update_sheet(df, exp_path, text):
    imageset = os.path.basename(exp_path).split('.')[0].split('_')[1:]
    if len(imageset) == 1: imageset = imageset[0]
    elif len(imageset) > 1: imageset = '_'.join(imageset)
    mask = df['ImageSet'] == imageset
    index = df.index[mask].tolist()[0]
    df.at[index, 'StimulusSet'] = text

for index, row in df.iterrows():
    if row['BrainScore']=='Y': list_of_bs_exp_names.append(row['ImageSet'])
    
experiment_file_paths = glob.glob(os.path.join(root_dir, '[exp]*', '*'))


def extract_number(filename):
    # Extract the number from the filename and return it as an integer
    match = re.search(r'\d+', filename)
    return int(match.group()) if match else 0

def update_exp_nwb(experiment_path, list_images_sorted, stimpath, count_1 = 0, check_image_order = True):

    combined_nwb = [x for x in os.listdir(experiment_path) if x.endswith('combined.nwb')]
    combined_nwb_test = [x for x in os.listdir(experiment_path) if x.endswith('combined_test.nwb')]
    combined_nwb_train = [x for x in os.listdir(experiment_path) if x.endswith('combined_train.nwb')]

    combined = False
    train    = False
    test     = False
    if len(combined_nwb) != 0: 
        # ------------------------------------------------------------------------------ 
        # Load combined nwb file.
        # ------------------------------------------------------------------------------ 
        io = NWBHDF5IO(os.path.join(experiment_path, combined_nwb[0]), "a") 
        combined_nwb = io.read()
        try: 
            combined_nwb.stimulus_template['StimulusSet']
            print('Simulus Set already exists in combined.')
            display(combined_nwb)
            combined = True
        except: pass

        n_stimuli = []
        for scratch in list(combined_nwb.scratch):
            if scratch.startswith('QualityCheckedPSTH'):
                n_stimuli.append(combined_nwb.scratch[scratch][:].shape[0])
        assert all(element == n_stimuli[0] for element in n_stimuli) == True, 'Number of Stimuli are not consistent over the PSTH!'
        n_stimuli = n_stimuli[0]
        
        # ------------------------------------------------------------------------------ 
        # Load combined_train nwb file.
        # ------------------------------------------------------------------------------ 
        io_train = NWBHDF5IO(os.path.join(experiment_path, combined_nwb_train[0]), "a") 
        combined_nwb_train = io_train.read()
        try: 
            combined_nwb_train.stimulus_template['StimulusSetTrain']
            print('Simulus Set already exists in train.')
            display(combined_nwb_train)
            train = True
        except: pass

        n_stimuli_train = []
        for scratch in list(combined_nwb_train.scratch):
            if scratch.startswith('QualityCheckedPSTH'):
                n_stimuli_train.append(combined_nwb_train.scratch[scratch][:].shape[0])
        assert all(element == n_stimuli_train[0] for element in n_stimuli_train) == True, 'Number of Stimuli are not consistent over the PSTH!'
        n_stimuli_train = n_stimuli_train[0]


        # ------------------------------------------------------------------------------ 
        # Load combined_test nwb file.
        # ------------------------------------------------------------------------------ 
        io_test = NWBHDF5IO(os.path.join(experiment_path, combined_nwb_test[0]), "a") 
        combined_nwb_test = io_test.read()
        try: 
            combined_nwb_test.stimulus_template['StimulusSetTest']
            print('Simulus Set already exists in test.')
            test = True
        except: pass

        n_stimuli_test = []
        for scratch in list(combined_nwb_test.scratch):
            if scratch.startswith('QualityCheckedPSTH'):
                n_stimuli_test.append(combined_nwb_test.scratch[scratch][:].shape[0])
        assert all(element == n_stimuli_test[0] for element in n_stimuli_test) == True, 'Number of Stimuli are not consistent over the PSTH!'
        n_stimuli_test = n_stimuli_test[0]
        print(n_stimuli, n_stimuli_train, n_stimuli_test)

        # ------------------------------------------------------------------------------ 
        # Create StimulusSets.
        # ------------------------------------------------------------------------------ 
        list_images = []
        list_images_train = []
        list_images_test = []
        for temp, image in enumerate(list_images_sorted):
            image_counter = temp
            if count_1 ==1: image_counter = image_counter+1

            expected_image_name = f'im{image_counter}'
            if check_image_order:
                if image.split(".")[0] != expected_image_name and image.split(".")[0] !=f'{image_counter}' and image.split(".")[0] !=f'image{image_counter}' and image.split(".")[0] !=f'im{image_counter}_scrambled' and image.split(".")[0] !=f'{image_counter}':
                        print(f'Image names do not increase with +1!!! Expected: {expected_image_name} or image{image_counter} or {image_counter}, Found: {image.split(".")[0]}')

            path = os.path.join(stimpath, image)
            img = Image.open(path)  # an example image

            nwb_image = RGBImage(
                name= image ,
                data=np.array(img.convert("RGB")),
                resolution=0.0,
                description= f"StimulusID = {temp}",
            )
            list_images.append(nwb_image)
            if temp < n_stimuli_train: list_images_train.append(nwb_image)
            if temp >= n_stimuli_train: list_images_test.append(nwb_image)
        
        assert n_stimuli == len(list_images), 'Number of Stimuli does not match number of Images!'

        # ------------------------------------------------------------------------------ 
        # Create nwb Images and append.
        # ------------------------------------------------------------------------------ 
        text=['']
        if combined == False: 
            all_images = Images(
                name=f'StimulusSet',
                images= list_images ,
                description= f"This list comprises images forming the stimulus set. \
                Each image in the list has a stimulusID in its description, corresponding to the stimulusID \
                in the PSTHs. If the filenames of these images contain numbers starting from 0 and increasing \
                sequentially, they correspond the StimulusID as well.",
            )
            try: 
                combined_nwb.add_stimulus_template(timeseries=all_images, use_sweep_table=False) 
                print(f"Added StimulusSet to combined nwb.")
                display(combined_nwb)
                io.write(combined_nwb)
                io.close()  
                text.append('StimulusSet added to combined nwb.')
            except Exception as error: 
                print("An error occurred:", error) 
                io.close()  
                text.append(error)

        if train == False:
            train_images = Images(
                name=f'StimulusSetTrain',
                images= list_images_train ,
                description= f"This list comprises images forming the stimulus set for this training set. \
                Each image in the list has a stimulusID in its description, corresponding to the stimulusID \
                in the PSTHs.",
            )
            try: 
                combined_nwb_train.add_stimulus_template(timeseries=train_images, use_sweep_table=False)  
                print(f"Added StimulusSet to train nwb.")
                display(combined_nwb_train)
                io_train.write(combined_nwb_train)
                io_train.close()    
                text.append('StimulusSet added to train nwb.')
            except Exception as error: 
                print("An error occurred:", error) 
                io_train.close() 
                text.append(error)

        if test == False:
            test_images = Images(
                name=f'StimulusSetTest',
                images= list_images_test ,
                description= f"This list comprises images forming the stimulus set for this test set. \
                Each image in the list has a stimulusID in its description, corresponding to the stimulusID \
                in the PSTHs.",
            )
            try: 
                combined_nwb_test.add_stimulus_template(timeseries=test_images, use_sweep_table=False) 
                print(f"Added StimulusSet to test nwb.")
                display(combined_nwb_test)
                io_test.write(combined_nwb_test)
                io_test.close()  
                text.append('StimulusSet added to test nwb.')
            except Exception as error: 
                print("An error occurred:", error)    
                io_test.close() 
                text.append(error)

        # ------------------------------------------------------------------------------ 
        # update excel.
        # ------------------------------------------------------------------------------ 
        if text==['']: text = 'StimulusSet created and added to nwb files.'
        # update_sheet(df, experiment_path, text)
        

def update_exp_nwb_movies(experiment_path, list_images_sorted, count_1, stimpath, check_image_order=True):

    combined_nwb = [x for x in os.listdir(experiment_path) if x.endswith('combined.nwb')]
    combined_nwb_test = [x for x in os.listdir(experiment_path) if x.endswith('combined_test.nwb')]
    combined_nwb_train = [x for x in os.listdir(experiment_path) if x.endswith('combined_train.nwb')]

    combined = False
    train    = False
    test     = False
    if len(combined_nwb) != 0: 
        # ------------------------------------------------------------------------------ 
        # Load combined nwb file.
        # ------------------------------------------------------------------------------ 
        io = NWBHDF5IO(os.path.join(experiment_path, combined_nwb[0]), "a") 
        combined_nwb = io.read()
        try: 
            combined_nwb.stimulus_template['StimulusSet']
            print('Simulus Set already exists in combined.')
            # display(combined_nwb)
            combined = True
        except: pass

        n_stimuli = []
        for scratch in list(combined_nwb.scratch):
            if scratch.startswith('QualityCheckedPSTH'):
                n_stimuli.append(combined_nwb.scratch[scratch][:].shape[0])
        assert all(element == n_stimuli[0] for element in n_stimuli) == True, 'Number of Stimuli are not consistent over the PSTH!'
        n_stimuli = n_stimuli[0]
        
        # ------------------------------------------------------------------------------ 
        # Load combined_train nwb file.
        # ------------------------------------------------------------------------------ 
        io_train = NWBHDF5IO(os.path.join(experiment_path, combined_nwb_train[0]), "a") 
        combined_nwb_train = io_train.read()
        try: 
            combined_nwb_train.stimulus_template['StimulusSetTrain']
            print('Simulus Set already exists in train.')
            # display(combined_nwb_train)
            train = True
        except: pass

        n_stimuli_train = []
        for scratch in list(combined_nwb_train.scratch):
            if scratch.startswith('QualityCheckedPSTH'):
                n_stimuli_train.append(combined_nwb_train.scratch[scratch][:].shape[0])
        assert all(element == n_stimuli_train[0] for element in n_stimuli_train) == True, 'Number of Stimuli are not consistent over the PSTH!'
        n_stimuli_train = n_stimuli_train[0]


        # ------------------------------------------------------------------------------ 
        # Load combined_test nwb file.
        # ------------------------------------------------------------------------------ 
        io_test = NWBHDF5IO(os.path.join(experiment_path, combined_nwb_test[0]), "a") 
        combined_nwb_test = io_test.read()
        try: 
            combined_nwb_test.stimulus_template['StimulusSetTest']
            print('Simulus Set already exists in test.')
            test = True
        except: pass

        n_stimuli_test = []
        for scratch in list(combined_nwb_test.scratch):
            if scratch.startswith('QualityCheckedPSTH'):
                n_stimuli_test.append(combined_nwb_test.scratch[scratch][:].shape[0])
        assert all(element == n_stimuli_test[0] for element in n_stimuli_test) == True, 'Number of Stimuli are not consistent over the PSTH!'
        n_stimuli_test = n_stimuli_test[0]
        print(n_stimuli, n_stimuli_train, n_stimuli_test)

        # ------------------------------------------------------------------------------ 
        # Create StimulusSets.
        # ------------------------------------------------------------------------------ 

        for temp, image in enumerate(list_images_sorted):
            if count_1 ==1: temp = temp+1

            expected_image_name = f'mv_{temp}'
            if check_image_order:
                if image.split(".")[0] != expected_image_name and image.split(".")[0] !=f'{temp}' and image.split(".")[0] !=f'mv{temp}' and image.split(".")[0] !=f'vid{temp}':
                        print(f'Image names do not increase with +1!!! Expected: {expected_image_name} or image{temp} or {temp}, Found: {image.split(".")[0]}')

        image_set_path = '/'.join(experiment_path.split('/')[:-1])
        combined_nwb_path = [x for x in os.listdir(experiment_path) if x.endswith('combined.nwb')]
        external_file = [os.path.relpath(os.path.join(image_set_path, 'VideoStimulusSet', movie_path), os.path.join(experiment_path, combined_nwb_path[0])) for movie_path in list_images_sorted]

        # ------------------------------------------------------------------------------ 
        # StimulusSet for combined.
        # ------------------------------------------------------------------------------ 
        StimulusMovieFile_combined = ImageSeries(
            name="StimulusSet",
            description=f"This list references external files linking to movies in the stimulus set. \
                Each movie has an associated 'starting_frame' field, repurposed to serve as the \
                stimulusID in PSTHs. Filenames with sequential numbers starting from 0 align with these \
                StimulusIDs. Note: The original function of 'starting_frame' is altered for this specific use case.",
            unit="n.a.",
            external_file=external_file,
            format="external", 
            rate=0.0, 
            starting_frame = np.linspace(0, n_stimuli-1, n_stimuli))
        
        # ------------------------------------------------------------------------------ 
        # StimulusSet for train.
        # ------------------------------------------------------------------------------ 
        StimulusMovieFile_train = ImageSeries(
            name="StimulusSetTrain",
            description="This list references external files linking to movies in the stimulus set. \
                Each movie has an associated 'starting_frame' field, repurposed to serve as the \
                stimulusID in PSTHs. Filenames with sequential numbers starting from 0 align with these \
                StimulusIDs. Note: The original function of 'starting_frame' is altered for this specific use case.",
            unit="n.a.",
            external_file=external_file[:n_stimuli_train],
            format="external", 
            rate=0.0, 
            starting_frame = np.linspace(0, n_stimuli_train-1, n_stimuli_train))
        
        # ------------------------------------------------------------------------------ 
        # StimulusSet for test.
        # ------------------------------------------------------------------------------ 
        StimulusMovieFile_test = ImageSeries(
            name="StimulusSetTest",
            description="This list references external files linking to movies in the stimulus set. \
                Each movie has an associated 'starting_frame' field, repurposed to serve as the \
                stimulusID in PSTHs. Filenames with sequential numbers starting from 0 align with these \
                StimulusIDs. Note: The original function of 'starting_frame' is altered for this specific use case.",
            unit="n.a.",
            external_file=external_file[n_stimuli_train:],
            format="external", 
            rate=0.0, 
            starting_frame = np.linspace(n_stimuli_train, n_stimuli-1, n_stimuli_test))
            
        assert n_stimuli == len(external_file), 'Number of Stimuli does not match number of Images!'

        # ------------------------------------------------------------------------------ 
        # Append to nwb files.
        # ------------------------------------------------------------------------------ 
        text=['']
        if combined == False: 
            try: 
                combined_nwb.add_stimulus_template(timeseries=StimulusMovieFile_combined, use_sweep_table=False) 
                print(f"Added StimulusSet to combined nwb.")
                display(combined_nwb)
                io.write(combined_nwb)
                io.close()  
                text.append('StimulusSet added to combined nwb.')
            except Exception as error: 
                print("An error occurred:", error) 
                io.close()  
                text.append(error)

        if train == False:
            try: 
                combined_nwb_train.add_stimulus_template(timeseries=StimulusMovieFile_train, use_sweep_table=False)  
                print(f"Added StimulusSet to train nwb.")
                display(combined_nwb_train)
                io_train.write(combined_nwb_train)
                io_train.close()    
                text.append('StimulusSet added to train nwb.')
            except Exception as error: 
                print("An error occurred:", error) 
                io_train.close() 
                text.append(error)

        if test == False:
            try: 
                combined_nwb_test.add_stimulus_template(timeseries=StimulusMovieFile_test, use_sweep_table=False) 
                print(f"Added StimulusSet to test nwb.")
                display(combined_nwb_test)
                io_test.write(combined_nwb_test)
                io_test.close()  
                text.append('StimulusSet added to test nwb.')
            except Exception as error: 
                print("An error occurred:", error)    
                io_test.close() 
                text.append(error)

        # ------------------------------------------------------------------------------ 
        # update excel.
        # ------------------------------------------------------------------------------ 
        if text==['']: text = 'StimulusSet created and added to nwb files.'
        # update_sheet(df, experiment_path, text)
        

for experiment_path in experiment_file_paths: 
    experiment_name =  "_".join(os.path.basename(experiment_path).split('.')[0].split('_')[1:])

    if experiment_name not in list_of_bs_exp_names: 
        continue 
    
    # ------------------------------------------------------------------------------ 
    # Run the manually defined ones from the exel file. 
    # ------------------------------------------------------------------------------ 
    # print('----------', df["StimulusSet"][df_index])
    df_index = np.where(df['ImageSet'].to_numpy() == experiment_name)[0][0]
    if type(df["StimulusSet"][df_index]) != str: continue 
    if  not df["StimulusSet"][df_index].startswith('/braintree'): continue 
    path = df["StimulusSet"][df_index]

    if not os.listdir(path)[0].endswith('.mp4'): 
        list_images = os.listdir(path)
        list_images_sorted = [x for x in sorted(list_images, key = extract_number) if not x.startswith('.')]

        if experiment_name == '1_shapes':
            print(experiment_name)
            update_exp_nwb(experiment_path, list_images_sorted, 1, path)

        if experiment_name =='gratingsAdap_s5':
            update_exp_nwb(experiment_path, list_images_sorted, 0, path)

        if experiment_name =='oasis900':
            def extract_integer(image_name):
                try:
                    return int(image_name[2:-4])
                except: pass

            csv_path = os.path.join('/', *path.split('/')[:-1], 'image_dicarlo_oasis900.csv')
            df_csv = pd.read_csv(csv_path)
            mapping = {extract_integer(row['image_file_name']): row['filename'] for index, row in df_csv.iterrows()}
            reverse_mapping = {v: k for k, v in mapping.items()}
            sorted_image_list = sorted(list_images_sorted, key=lambda x: reverse_mapping.get(x, float('inf')))
            update_exp_nwb(experiment_path, sorted_image_list, 0, path, check_image_order=False)

        if experiment_name =='oasis900rotated':
            """
            According to the mwel file, the stimulus is organized as first 900 ori and then 900 rot.
            file_path = os.path.join('/', *path.split('/')[:-1],'image_set_definition_oriandrotated.mwel' ) 
            with open(file_path, 'r') as file:
                mwel_content = file.read()
            print(mwel_content)
            
            """

            list_images_sorted_ori = [x for x in list_images_sorted if x.startswith('im_ori')]
            list_images_sorted_rot = [x for x in list_images_sorted if x.startswith('im_rot')]
            list_images_sorted = list_images_sorted_ori + list_images_sorted_rot
            update_exp_nwb(experiment_path, list_images_sorted, 0, path, check_image_order=False)

        if experiment_name =='shapenet360':
            update_exp_nwb(experiment_path, list_images_sorted, 1, path)

    
    if os.listdir(path)[0].endswith('.mp4'):
        list_movies = os.listdir(path)
        list_movies_sorted = [x for x in sorted(list_movies, key = extract_number) if not x.startswith('.')]

        # ------------------------------------------------------------------------------ 
        # Copy movies into experiment file.
        # ------------------------------------------------------------------------------ 
        image_set_path = '/'.join(experiment_path.split('/')[:-1])
        try: os.mkdir(os.path.join(image_set_path, 'VideoStimulusSet'))
        except: pass
        for movie in list_movies_sorted:
            try: shutil.copy2(os.path.join(path, movie), os.path.join(image_set_path, 'VideoStimulusSet'))
            except: pass
        
        # ------------------------------------------------------------------------------ 
        # Update nwb files.
        # ------------------------------------------------------------------------------ 
        print(experiment_name)
        if experiment_name == 'motionset1' or experiment_name == 'moca' or experiment_name == 'afv'  or experiment_name == 'faceemovids' or experiment_name.startswith('gratingsAdap_'):
            update_exp_nwb_movies(experiment_path, list_movies_sorted, 0, path)
        elif experiment_name == 'gestalt' or experiment_name == 'object_relations' or experiment_name == 'Co3D':
            update_exp_nwb_movies(experiment_path, list_movies_sorted, 1, path)
        




    

In [ ]:
list_of_bs_exp_names = []
df = pd.read_excel( os.path.dirname(cwd)+'/pico_inventory.xlsx' , sheet_name='Sheet2')
df['StimulusSet'] = ''

def update_sheet(df, exp_path, text):
    imageset = os.path.basename(exp_path).split('.')[0].split('_')[1:]
    if len(imageset) == 1: imageset = imageset[0]
    elif len(imageset) > 1: imageset = '_'.join(imageset)
    mask = df['ImageSet'] == imageset
    index = df.index[mask].tolist()[0]
    df.at[index, 'StimulusSet'] = text

for index, row in df.iterrows():
    if row['BrainScore']=='Y': list_of_bs_exp_names.append(row['ImageSet'])
    
experiment_file_paths = glob.glob(os.path.join(root_dir, '[exp]*', '*'))
stimulus_dir          = '/braintree/data2/active/users/sgouldin/experiments-codebase'
stimuli_names         = os.listdir(stimulus_dir)



def extract_number(filename):
    # Extract the number from the filename and return it as an integer
    match = re.search(r'\d+', filename)
    return int(match.group()) if match else 0

def update_exp_nwb(experiment_path, list_images_sorted, sourcefolder):

    combined_nwb = [x for x in os.listdir(experiment_path) if x.endswith('combined.nwb')]
    combined_nwb_test = [x for x in os.listdir(experiment_path) if x.endswith('combined_test.nwb')]
    combined_nwb_train = [x for x in os.listdir(experiment_path) if x.endswith('combined_train.nwb')]

    combined = False
    train    = False
    test     = False
    if len(combined_nwb) != 0: 
        # ------------------------------------------------------------------------------ 
        # Load combined nwb file.
        # ------------------------------------------------------------------------------ 
        io = NWBHDF5IO(os.path.join(experiment_path, combined_nwb[0]), "a") 
        combined_nwb = io.read()
        try: 
            combined_nwb.stimulus_template['StimulusSet']
            # display(combined_nwb)
            print('Simulus Set already exists in combined.')
            combined = True
        except: pass

        n_stimuli = []
        for scratch in list(combined_nwb.scratch):
            if scratch.startswith('QualityCheckedPSTH'):
                n_stimuli.append(combined_nwb.scratch[scratch][:].shape[0])
        assert all(element == n_stimuli[0] for element in n_stimuli) == True, 'Number of Stimuli are not consistent over the PSTH!'
        n_stimuli = n_stimuli[0]
        
        # ------------------------------------------------------------------------------ 
        # Load combined_train nwb file.
        # ------------------------------------------------------------------------------ 
        io_train = NWBHDF5IO(os.path.join(experiment_path, combined_nwb_train[0]), "a") 
        combined_nwb_train = io_train.read()
        try: 
            combined_nwb_train.stimulus_template['StimulusSetTrain']
            # display(combined_nwb_train)
            print('Simulus Set already exists in train.')
            train = True
        except: pass

        n_stimuli_train = []
        for scratch in list(combined_nwb_train.scratch):
            if scratch.startswith('QualityCheckedPSTH'):
                n_stimuli_train.append(combined_nwb_train.scratch[scratch][:].shape[0])
        assert all(element == n_stimuli_train[0] for element in n_stimuli_train) == True, 'Number of Stimuli are not consistent over the PSTH!'
        n_stimuli_train = n_stimuli_train[0]


        # ------------------------------------------------------------------------------ 
        # Load combined_test nwb file.
        # ------------------------------------------------------------------------------ 
        io_test = NWBHDF5IO(os.path.join(experiment_path, combined_nwb_test[0]), "a") 
        combined_nwb_test = io_test.read()
        try: 
            combined_nwb_test.stimulus_template['StimulusSetTest']
            # display(combined_nwb_test)
            print('Simulus Set already exists in test.')
            test = True
        except: pass

        n_stimuli_test = []
        for scratch in list(combined_nwb_test.scratch):
            if scratch.startswith('QualityCheckedPSTH'):
                n_stimuli_test.append(combined_nwb_test.scratch[scratch][:].shape[0])
        assert all(element == n_stimuli_test[0] for element in n_stimuli_test) == True, 'Number of Stimuli are not consistent over the PSTH!'
        n_stimuli_test = n_stimuli_test[0]
        print(n_stimuli, n_stimuli_train, n_stimuli_test)

        # ------------------------------------------------------------------------------ 
        # Create StimulusSets.
        # ------------------------------------------------------------------------------ 
        list_images = []
        list_images_train = []
        list_images_test = []
        for temp, image in enumerate(list_images_sorted):
            expected_image_name = f'im{temp}'
            if image.split(".")[0] != expected_image_name and image.split(".")[0] !=f'{temp}' and image.split(".")[0] !=f'image{temp}':
                    print(f'Image names do not increase with +1!!! Expected: {expected_image_name} or image{temp} or {temp}, Found: {image.split(".")[0]}')

            path = os.path.join(stimulus_dir, stim_name[0], sourcefolder, image)
            img = Image.open(path)  # an example image

            nwb_image = RGBImage(
                name= image ,
                data=np.array(img.convert("RGB")),
                resolution=0.0,
                description= f"StimulusID = {temp}",
            )
            list_images.append(nwb_image)
            if temp < n_stimuli_train: list_images_train.append(nwb_image)
            if temp >= n_stimuli_train: list_images_test.append(nwb_image)
        
        assert n_stimuli == len(list_images), 'Number of Stimuli does not match number of Images!'

        # ------------------------------------------------------------------------------ 
        # Create nwb Images and append.
        # ------------------------------------------------------------------------------ 
        text=['']
        if combined == False: 
            all_images = Images(
                name=f'StimulusSet',
                images= list_images ,
                description= f"This list comprises images forming the stimulus set. \
                Each image in the list has a stimulusID in its description, corresponding to the stimulusID \
                in the PSTHs. If the filenames of these images contain numbers starting from 0 and increasing \
                sequentially, they correspond the StimulusID as well.",
            )
            try: 
                combined_nwb.add_stimulus_template(timeseries=all_images, use_sweep_table=False) 
                io.write(combined_nwb)
                io.close()  
                text.append('StimulusSet added to combined nwb.')
            except Exception as error: 
                print("An error occurred:", error) 
                io.close()  
                text.append(error)

        if train == False:
            train_images = Images(
                name=f'StimulusSetTrain',
                images= list_images_train ,
                description= f"This list comprises images forming the stimulus set for this training set. \
                Each image in the list has a stimulusID in its description, corresponding to the stimulusID \
                in the PSTHs.",
            )
            try: 
                combined_nwb_train.add_stimulus_template(timeseries=train_images, use_sweep_table=False)  
                io_train.write(combined_nwb_train)
                io_train.close()    
                text.append('StimulusSet added to train nwb.')
            except Exception as error: 
                print("An error occurred:", error) 
                io_train.close() 
                text.append(error)

        if test == False:
            test_images = Images(
                name=f'StimulusSetTest',
                images= list_images_test ,
                description= f"This list comprises images forming the stimulus set for this test set. \
                Each image in the list has a stimulusID in its description, corresponding to the stimulusID \
                in the PSTHs.",
            )
            try: 
                combined_nwb_test.add_stimulus_template(timeseries=test_images, use_sweep_table=False) 
                io_test.write(combined_nwb_test)
                io_test.close()  
                text.append('StimulusSet added to test nwb.')
            except Exception as error: 
                print("An error occurred:", error)    
                io_test.close() 
                text.append(error)

        # ------------------------------------------------------------------------------ 
        # update excel.
        # ------------------------------------------------------------------------------ 
        if text==['']: text = 'StimulusSet created and added to nwb files.'
        update_sheet(df, experiment_path, text)
        

for experiment_path in experiment_file_paths: 
    experiment_name =  "_".join(os.path.basename(experiment_path).split('.')[0].split('_')[1:])

    print('________________________________________________________________________________')
    if experiment_name not in list_of_bs_exp_names: 
        continue 
    
    # ------------------------------------------------------------------------------ 
    # Define some files manually.
    # ------------------------------------------------------------------------------ 
    if experiment_name == 'domain-transfer-2023':
        stim_name = [x for x in stimuli_names if x.endswith('domain_transfer') and not x.startswith('.')]
    elif experiment_name == 'HVM-var6-2023':
        stim_name = [x for x in stimuli_names if x.endswith('HVM_var6') and not x.startswith('.')]
    elif experiment_name.startswith('gratingsAdap_'):
        stim_name = [x for x in stimuli_names if x.endswith('gratingsAdap') and not x.startswith('.')]
    elif experiment_name.startswith('gestalt'):
        stim_name = [x for x in stimuli_names if x.endswith('Gestalt') and not x.startswith('.')]
    elif experiment_name.startswith('object_relations'):
        stim_name = [x for x in stimuli_names if x.endswith('ObjectRelationships') and not x.startswith('.')]
    elif experiment_name.startswith('1_shapes'):
        stim_name = [x for x in stimuli_names if x.endswith('shapes') and not x.startswith('.')]
    # elif experiment_name.startwith('food'):
    #     stim_name = [x for x in stimuli_names if x.endswith('Food') and not x.startswith('.')]
    else:
        stim_name = [x for x in stimuli_names if x.endswith(experiment_name) and not x.startswith('.')]

    if len(stim_name)==0: 
        print(f'    No Stim found: {experiment_name}') 
        update_sheet(df, experiment_path, 'No StimulusSet found.')
        continue

    files_starting_with_vid = [file for file in os.listdir(os.path.join(stimulus_dir, stim_name[0])) if file.startswith('vid')]


    # ------------------------------------------------------------------------------ 
    # Find 'images' folder and update stimulus in nwb file.
    # ------------------------------------------------------------------------------ 
    if 'images' in os.listdir(os.path.join(stimulus_dir, stim_name[0])): 
        list_images = os.listdir(os.path.join(stimulus_dir, stim_name[0], 'images'))
        list_images_sorted = [x for x in sorted(list_images, key = extract_number) if not x.startswith('.')]
        df_index = np.where(df['ImageSet'].to_numpy() == experiment_name)[0][0]
        df.at[df_index, "StimulusSet"] = os.path.join(stimulus_dir, stim_name[0], 'images')

        print('images:', experiment_name, list_images_sorted)
        update_exp_nwb(experiment_path, list_images_sorted, os.path.join(stimulus_dir, stim_name[0], 'images'))
        
    
    # ------------------------------------------------------------------------------ 
    # Find 'vid...' folder and update stimulus in nwb file.
    # ------------------------------------------------------------------------------ 
    elif len(files_starting_with_vid) > 0: 
        print(files_starting_with_vid)
        list_videos = os.listdir(os.path.join(stimulus_dir, stim_name[0], files_starting_with_vid[0]))
        list_videos_sorted = [x for x in sorted(list_videos, key = extract_number) if not x.startswith('.')]
        
        print('videos', stim_name[0], list_videos_sorted)
        df_index = np.where(df['ImageSet'].to_numpy() == experiment_name)[0][0]
        df.at[df_index, "StimulusSet"] = os.path.join(stimulus_dir, stim_name[0], 'videos')

        # update_exp_nwb(experiment_path, list_videos_sorted, 'videos')

    # ------------------------------------------------------------------------------ 
    # Manually find folders for gratingsAdap
    # ------------------------------------------------------------------------------ 
    elif experiment_name.startswith('gratingsAdap_'):
        season = experiment_name.split('_')[-1][-1]
        file = f'season{season}'
        list_videos = os.listdir(os.path.join(stimulus_dir, stim_name[0], file))
        list_videos_sorted = [x for x in sorted(list_videos, key = extract_number) if not x.startswith('.')]
        print('videos', stim_name[0], list_videos_sorted)
        df_index = np.where(df['ImageSet'].to_numpy() == experiment_name)[0][0]
        df.at[df_index, "StimulusSet"] = os.path.join(stimulus_dir, stim_name[0], file)
    
    # ------------------------------------------------------------------------------ 
    # Manually find folders for object_relations
    # ------------------------------------------------------------------------------ 
    elif experiment_name.startswith('object_relations'):
        files_starting_with_vid = [file for file in os.listdir(os.path.join(stimulus_dir, stim_name[0])) if file.startswith('mworks')]
        list_videos = os.listdir(os.path.join(stimulus_dir, stim_name[0], files_starting_with_vid[0]))
        list_videos_sorted = [x for x in sorted(list_videos, key = extract_number) if not x.startswith('.')]
        
        print('videos', stim_name[0], list_videos_sorted)
        df_index = np.where(df['ImageSet'].to_numpy() == experiment_name)[0][0]
        df.at[df_index, "StimulusSet"] = os.path.join(stimulus_dir, stim_name[0], 'videos')

    
    else: print(f'  No Images or Videos found for {stim_name[0]}')
    

# Update Sheet 2
# xls = pd.ExcelFile(f'{os.path.dirname(cwd)}/pico_inventory.xlsx')
# sheets = {sheet: xls.parse(sheet) for sheet in xls.sheet_names}

# sheets['Sheet2'] = df  

# with pd.ExcelWriter(f'{os.path.dirname(cwd)}/pico_inventory.xlsx', engine='openpyxl', mode='w') as writer:
#     for sheet_name, sheet_df in sheets.items():
#         sheet_df.to_excel(writer, sheet_name=sheet_name, index=False)      
    